# STEP 1 : Data Analysis

In [1]:
import math
import IPython
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn import preprocessing, decomposition, discriminant_analysis, tree

pd.options.display.max_columns = None

In [2]:
raw_train_credit_application = pd.read_csv('data/application_train.csv')
raw_test_credit_application = pd.read_csv('data/application_test.csv')

raw_bureau = pd.read_csv('data/bureau.csv')
raw_bureau_balance = pd.read_csv('data/bureau_balance.csv')
raw_credit_card_balance = pd.read_csv('data/credit_card_balance.csv')
raw_installments_payments = pd.read_csv('data/installments_payments.csv')
raw_pos_cash_balance = pd.read_csv('data/pos_cash_balance.csv')
raw_previous_application = pd.read_csv('data/previous_application.csv')

In [8]:
def make_summary(raw_df, name):
    non_features = ['SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV']
    numerical_features = []
    categorical_features = []
    flag_features = []
    other_features = []

    for feature in raw_df.columns:
        if feature in non_features:
            continue
        elif raw_df.dtypes[feature] in [np.dtype('int64'), np.dtype('float64')]:
            unique_values = raw_df[feature].unique()
            unique_values = unique_values[~np.isnan(unique_values)]
            if set(unique_values).issubset(set([0, 1])) or \
                set(unique_values).issubset(set([0.0, 1.1])): 
                flag_features.append(feature)
            else:
                numerical_features.append(feature)
        elif raw_df.dtypes[feature] in [np.dtype('object')]:
            categorical_features.append(feature)
        else:
            other_features.append(feature)
    
    print(name)
    print(numerical_features)
    print(categorical_features)
    print(flag_features)
    print(other_features)
    print('\n')
            
#     preproc_df = pd.get_dummies(raw_df, columns=categorical_features)
#     preproc_all_features = preproc_df.columns.values

#     d = dict()

#     if 'SK_ID_BUREAU' in raw_df.columns:
#         d['SK_ID_BUREAU'] = 'count'
#     elif 'SK_ID_PREV' in raw_df.columns:
#         d['SK_ID_PREV'] = 'count'

#     for feature in numerical_features:
#         d[feature] = np.mean

#     onehot_features = []
#     for feature in (categorical_features + flag_features):
#         for i in preproc_all_features:
#             if feature in i:
#                 onehot_features.append(i)

#     for feature in onehot_features:
#         d[feature] = np.sum

#     summary_df = preproc_df.groupby(['SK_ID_CURR'], as_index=False).agg(d)

#     new_feature_names = []
#     for feature_name in summary_df.columns.values:
#         if feature_name == 'SK_ID_CURR':
#             new_feature_names.append(feature_name)
#         elif feature_name == 'SK_ID_PREV':
#             new_feature_names.append('PREV_count')
#         elif feature_name == 'SK_ID_BUREAU':
#             new_feature_names.append('BUREAU_count')
#         else:
#             new_feature_names.append('PREV_' + str(feature_name.lower()))

#     summary_df.columns = new_feature_names
    
#     summary_df.to_pickle('data/summary_%s.pkl' % name)

In [10]:
raw_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [11]:
raw_credit_card_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [9]:
make_summary(raw_bureau, name='bureau')
make_summary(raw_credit_card_balance, name='credit_card_balance')
make_summary(raw_installments_payments, name='installments_payments')
make_summary(raw_pos_cash_balance, name='pos_cash_balance')
make_summary(raw_previous_application, name='previous_application')

bureau
['DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY']
['CREDIT_ACTIVE', 'CREDIT_CURRENCY', 'CREDIT_TYPE']
[]
[]


credit_card_balance
['MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY', 'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', 'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM', 'SK_DPD', 'SK_DPD_DEF']
['NAME_CONTRACT_STATUS']
[]
[]


installments_payments
['NUM_INSTALMENT_VERSION', 'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'AMT_PA

In [13]:
def collect_data(raw_credit_application):
    new_df = raw_credit_application.copy()
    new_df = pd.merge(left=new_df, right=pd.read_pickle('data/summary_bureau.pkl'), left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='outer')
    new_df = pd.merge(left=new_df, right=pd.read_pickle('data/summary_credit_card_balance.pkl'), left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='outer')
    new_df = pd.merge(left=new_df, right=pd.read_pickle('data/summary_installments_payments.pkl'), left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='outer')
    new_df = pd.merge(left=new_df, right=pd.read_pickle('data/summary_pos_cash_balance.pkl'), left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='outer')
    new_df = pd.merge(left=new_df, right=pd.read_pickle('data/summary_previous_application.pkl'), left_on='SK_ID_CURR', right_on='SK_ID_CURR', how='outer')
    
    return new_df

In [14]:
complete_train_credit_application = collect_data(raw_train_credit_application)
complete_test_credit_application = collect_data(raw_test_credit_application)

In [15]:
complete_train_credit_application.to_pickle('data/complete_train_credit_application.pkl')
complete_test_credit_application.to_pickle('data/complete_test_credit_application.pkl')

In [16]:
complete_train_credit_application.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BUREAU_count,PREV_days_credit,PREV_credit_day_overdue,PREV_days_credit_enddate,PREV_days_enddate_fact,PREV_amt_credit_max_overdue,PREV_cnt_credit_prolong,PREV_amt_credit_sum,PREV_amt_credit_sum_debt,PREV_amt_credit_sum_limit,PREV_amt_credit_sum_overdue,PREV_days_credit_update,PREV_amt_annuity_x,PREV_credit_active_active,PREV_credit_active_bad debt,PREV_credit_active_closed,PREV_credit_active_sold,PREV_credit_currency_currency 1,PREV_credit_currency_currency 2,PREV_credit_currency_currency 3,PREV_credit_currency_currency 4,PREV_credit_type_another type of loan,PREV_credit_type_car loan,PREV_credit_type_cash loan (non-earmarked),PREV_credit_type_consumer credit,PREV_credit_type_credit card,PREV_credit_type_interbank credit,PREV_credit_type_loan for business development,PREV_credit_type_loan for purchase of shares (margin lending),PREV_credit_type_loan for the purchase of equipment,PREV_credit_type_loan for working capital replenishment,PREV_credit_type_microloan,PREV_credit_type_mobile operator loan,PREV_credit_type_mortgage,PREV_credit_type_real estate loan,PREV_credit_type_unknown type of loan,PREV_count_x,PREV_months_balance_x,PREV_amt_balance,PREV_amt_credit_limit_actual,PREV_amt_drawings_atm_current,PREV_amt_drawings_current,PREV_amt_drawings_other_current,PREV_amt_drawings_pos_current,PREV_amt_inst_min_regularity,PREV_amt_payment_current,PREV_amt_payment_total_current,PREV_amt_receivable_principal,PREV_amt_recivable,PREV_amt_total_receivable,PREV_cnt_drawings_atm_current,PREV_cnt_drawings_current,PREV_cnt_drawings_other_current,PREV_cnt_drawings_pos_current,PREV_cnt_instalment_mature_cum,PREV_sk_dpd_x,PREV_sk_dpd_def_x,PREV_name_contract_status_active_x,PREV_name_contract_status_approved_x,PREV_name_contract_status_completed_x,PREV_name_contract_status_demand_x,PR